Importación de Librerías

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

torch: Es la biblioteca principal de PyTorch que proporciona tensores y otras funcionalidades esenciales.
nn: Es el módulo de redes neuronales de PyTorch, que contiene todos los componentes para construir redes neuronales.
DataLoader: Clase que permite cargar los datos de manera eficiente en mini-lotes, con opciones de barajado y carga paralela.
datasets: Módulo que proporciona acceso a varios datasets comunes.
ToTensor: Transforma imágenes en formato PIL o numpy arrays a tensores de PyTorch.
Descarga del Dataset MNIST

In [ ]:
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)


datasets.MNIST: Clase para descargar y cargar el dataset MNIST.
root="data": Directorio donde se almacenarán los datos.
train=False/True: Indica si se deben cargar los datos de prueba (False) o de entrenamiento (True).
download=True: Descarga los datos si no están disponibles en el directorio especificado.
transform=ToTensor(): Aplica la transformación para convertir las imágenes a tensores de PyTorch.
Definición de DataLoader

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)


DataLoader: Clase que proporciona una manera conveniente de cargar datos en mini-lotes.
batch_size=64: Número de muestras por mini-lote.
shuffle=True/False: Mezcla los datos cada vez que se cargan (True para entrenamiento, False para prueba).
Imprimir la Forma de los Datos y los Targets

In [ ]:
print(training_data.data.shape)
print(training_data.targets.shape)


training_data.data.shape: Imprime la forma del tensor de datos de entrenamiento.
training_data.targets.shape: Imprime la forma del tensor de etiquetas de entrenamiento.
Definición de la Arquitectura de la Red Neuronal Mejorada

In [ ]:
class ImprovedNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


class ImprovedNeuralNetwork(nn.Module): Define la clase de la red neuronal, heredando de nn.Module.
init(self): Constructor de la clase.

super().__init__(): Llama al constructor de la clase base nn.Module.
self.flatten = nn.Flatten(): Define una capa que aplana las imágenes de 28x28 píxeles a un vector de 784 elementos.
self.linear_relu_stack = nn.Sequential(...): Define una secuencia de capas:
nn.Linear(28*28, 1024): Capa lineal con 784 entradas y 1024 salidas.
nn.BatchNorm1d(1024): Normalización por lotes para 1024 características, que ayuda a estabilizar y acelerar el entrenamiento.
nn.ReLU(): Función de activación ReLU.
nn.Dropout(0.5): Dropout con una tasa del 50%, que ayuda a prevenir el sobreajuste.
nn.Linear(1024, 512): Capa lineal con 1024 entradas y 512 salidas.
nn.BatchNorm1d(512): Normalización por lotes para 512 características.
nn.ReLU(): Función de activación ReLU.
nn.Dropout(0.5): Dropout con una tasa del 50%.
nn.Linear(512, 256): Capa lineal con 512 entradas y 256 salidas.
nn.BatchNorm1d(256): Normalización por lotes para 256 características.
nn.ReLU(): Función de activación ReLU.
nn.Linear(256, 10): Capa lineal con 256 entradas y 10 salidas, correspondiente a las 10 clases del dataset MNIST.
forward(self, x): Método que define el flujo de datos a través de la red.

x = self.flatten(x): Aplana la imagen de entrada.
logits = self.linear_relu_stack(x): Pasa los datos a través de la secuencia de capas.
return logits: Devuelve los logits (salidas no normalizadas de la red).
Preparación del Modelo y el Dispositivo

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


loss_fn: Define la función de pérdida como la pérdida de entropía cruzada, adecuada para problemas de clasificación.
optimizer: Define el optimizador Adam:
model.parameters(): Pasa los parámetros del modelo al optimizador.
lr=1e-3: Establece la tasa de aprendizaje a 0.001.
Scheduler para Disminuir el Learning Rate

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


scheduler: Define un scheduler para ajustar el learning rate:
step_size=5: Disminuye el learning rate cada 5 épocas.
gamma=0.1: Factor por el que se multiplica el learning rate (0.1).
Función de Entrenamiento

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_num, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_num % 100 == 0:
            loss, current = loss.item(), (batch_num + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


train(dataloader, model, loss_fn, optimizer): Función para entrenar el modelo.
size = len(dataloader.dataset): Obtiene el tamaño del dataset.
model.train(): Pone el modelo en modo de entrenamiento.
for batch_num, (X, y) in enumerate(dataloader): Itera sobre los mini-lotes del dataloader.
X, y = X.to(device), y.to(device): Mueve los datos y las etiquetas al dispositivo seleccionado.
pred = model(X): Realiza predicciones para el mini-lote.
loss = loss_fn(pred, y): Calcula la pérdida entre las predicciones y las etiquetas reales.
optimizer.zero_grad(): Resetea los gradientes de los parámetros del modelo.
loss.backward(): Calcula los gradientes de la pérdida con respecto a los parámetros del modelo.
optimizer.step(): Actualiza los parámetros del modelo según los gradientes.
if batch_num % 100 == 0: Cada 100 lotes, imprime el progreso.

Todo el código junto ahora.

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Descarga el dataset MNIST
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Define el dataloader
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

# Imprime la forma de los datos y los targets
print(training_data.data.shape)
print(training_data.targets.shape)

# Define la arquitectura de la red neuronal mejorada
class ImprovedNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = ImprovedNeuralNetwork()
device = (
    "cuda" if torch.cuda.is_available() 
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = model.to(device)
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Scheduler para disminuir el learning rate
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_num, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_num % 100 == 0:
            loss, current = loss.item(), (batch_num + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    accuracy = correct * 100
    print(f"Test Error: \n Accuracy: {accuracy:>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return accuracy, test_loss

# Implementar Early Stopping
best_accuracy = 0
epochs = 20
patience = 3
trigger_times = 0

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    accuracy, avg_loss = test(test_dataloader, model, loss_fn)
    
    # Actualizar el scheduler
    scheduler.step()
    
    # Early Stopping
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        trigger_times = 0
    else:
        trigger_times += 1
        print(f'Early stopping trigger times: {trigger_times}')
        
        if trigger_times >= patience:
            print('Early stopping!')
            break

print(f"Best Accuracy: {best_accuracy:>0.1f}%")
print("Done!")



torch.Size([60000, 28, 28])
torch.Size([60000])
Using cpu device
ImprovedNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=1024, out_features=512, bias=True)
    (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=512, out_features=256, bias=True)
    (9): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Linear(in_features=256, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.460386  [   64/60000]
loss: 0.164824  [ 6464/60000]
loss: 0.144975  [12864/60000]
loss: 0.287252  [19264/60000]
loss: 0.228738  [